In [1]:
!python3 --version
!pip3 list

Python 3.8.10
Package                 Version
----------------------- --------------------
absl-py                 0.13.0
anyio                   3.2.1
appdirs                 1.4.3
apturl                  0.5.2
argon2-cffi             20.1.0
astunparse              1.6.3
async-generator         1.10
attrs                   21.2.0
autopep8                1.5.7
Babel                   2.9.1
backcall                0.2.0
beautifulsoup4          4.9.3
bitarray                2.3.3
bleach                  3.3.0
blinker                 1.4
Brlapi                  0.7.0
cachetools              4.2.2
certifi                 2019.11.28
cffi                    1.14.5
chardet                 3.0.4
chrome-gnome-shell      0.0.0
clang                   5.0
Click                   7.0
colorama                0.4.3
command-not-found       0.3
cryptography            2.8
cupshelpers             1.0
cycler                  0.10.0
dbus-python             1.2.16
decorator               5.0.9
defer      

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import modules.model as Model
import pandas as pd

# Train test split
* Phần này tập trung chủ yếu vào việc chuẩn bị dữ liệu trước khi tiến hành đào tạo hàng loạt các model.

Đọc dữ liệu

In [4]:
data = pd.read_csv("./data/normalize_reviews.csv").fillna("")[['raw_comment', 'normalize_comment', 'label']]

data.head()

,raw_comment,normalize_comment,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,0
2,Ko có lắc tay như hình,không có lắc tay như hình,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,0


Do **sklearn** hoạt động tốt nhất trên text data có dạng mã hóa là **NFC**, nên trước tiên ta sẽ chuyển toàn bộ các feature có `dtype` là `str` được biểu diễn dưới dạng mã hóa NFC.

In [5]:
data = Model.textNFxformat(data, ['raw_comment', 'normalize_comment'], 'NFD')

data.head()

,raw_comment,normalize_comment,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,0
2,Ko có lắc tay như hình,không có lắc tay như hình,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,0


Các vectorizer object của **sklearn** mặc định chúng sẽ xóa toàn bộ các **punctuation** [kí tự đặc biệt] trong input truyền vào. Như vậy các emoji của ta sẽ bị xóa toàn bộ khi transform vectorizing. Như vậy, ta sẽ không lưu chúng dưới dạng các punctuation mà dùng decode của chúng - sau đó ta tạo feature `emoji_decode` để lưu chúng.  

In [6]:
data['emoji_decode'] = data['raw_comment'].apply(lambda s: Model.expandEmojisDecode(s))
data = data[['raw_comment', 'normalize_comment', 'emoji_decode', 'label']]

data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,cry,0


Tiến hành chọn các mẫu có feature `emoji_decode` không phải là chuổi rỗng và lưu vào biến `emoji_data`.

In [7]:
emoji_data = data[data['emoji_decode'] != ""].reset_index(drop=True)

emoji_data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,cry,0
1,Giao sai màu sai size có 1 dép lông size 3...,giao sai màu sai size có dép lông size à ...,cursing_face cursing_face cursing_face,0
2,Đơn hàng đã thanh toán airpay rồi mà sh...,đơn hàng thanh toán mà giao cho người l...,roll_eyes,0
3,Áo croptop freesize rộng với mình \nMìn...,áo rộng mình kg,thumbsup thumbsup thumbsup thumbsup thumbsup t...,0
4,Khá buồn . Đặt 2 cái đều không chạy ...,khá buồn đặt cái đều không chạy giơ...,female_sign shrug,0


Giờ thì ta sẽ tiến hành chia `emoji_data` thành training data và test data với size của test data là 15%, sau đó ta lưu chúng dưới dạng file **.csv**.

In [8]:
Model.dataSplitSaved(emoji_data, 0.2, "./data/emoji_data")

📢 Your dataset has saved at ./data/emoji_data.


Ta cũng sẽ tiến hành chia training data và test data cho 

In [9]:
data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,cry,0


In [10]:
Model.dataSplitSaved(data, 0.2, "./data/data")

📢 Your dataset has saved at ./data/data.
